In [30]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler #표준화(평균을 기준)
from sklearn.preprocessing import MinMaxScaler   #정규화(정해진 규칙 기준, min-max)
from sklearn.datasets import load_iris

import time
from IPython.display import Image
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score
# cross_val_score: 다양한 샘플링을 해서 여러번 테스트 --> 평균치를 구해본다.!
from sklearn.metrics import *

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder
from matplotlib import rc, font_manager
import matplotlib.font_manager as fm

# 디렉토리 및 파일 이름에 맞추어 변경
font_location = '/Users/syshin/Desktop/Himedia/Himedia_project/NanumGothic.ttf'

font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)

In [31]:
# data12 - 기계마모
# data14 - 앙상블voting, 서울시구별영역나누기
# 위 코드들 이용하여 머신러닝 모델 만들기

In [32]:
df=pd.read_csv('국내여행_병합.csv')

In [33]:
df.columns

Index(['Unnamed: 0', 'year', 'target_area1', 'target_area2', 'target_area3',
       'target_area4', 'target_area5', 'target_area6', 'target_area7',
       'target_area8', 'target_area9', 'target_area10', 'target_area11',
       'no_of_place', 'sleepnights_1', 'sleepnights_2', 'sleepnights_3',
       'sleepnights_4', 'sleepnights_5', 'sleepnights_6', 'sleepnights_7',
       'sleepnights_8', 'accommodation_1', 'accommodation_2',
       'accommodation_3', 'number_of_ppl', 'number_of_children',
       'cost_total_trip', 'satisfied', 'sex', 'age', 'month', 'living_area',
       'transportation1', 'transportation2', 'transportation3', 'reason1',
       'reason2', 'reason3', 'relationship1', 'relationship2', 'relationship3',
       'target'],
      dtype='object')

In [34]:
# sleepnights_1의 컬럼명을 sleepnights로 변경 
df.rename(columns={'sleepnights_1':'sleepnights'}, inplace=True)
# relationship1의 컬럼명을 relationship으로 변경
df.rename(columns={'relationship1':'relationship'}, inplace=True)

In [35]:
# unnamed, year, no_of_place, target_area_1~11, sleepnights_2~8, satisfied, relationship2~3 제외
data=df[['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']]
target=df['target']

In [36]:
X_train, X_test, y_train, y_test=train_test_split(
    data,
    target,
    test_size=0.2,
    random_state=77
)

In [37]:
knn=KNeighborsClassifier()
svm=SVC()
dt=DecisionTreeClassifier()
model_list=[knn, svm, dt]

In [38]:
def print_cv(model, train, target, k):
    scores=cross_val_score(model, train, target, cv=k)
    print('----교차검증(cv)결과 리스트----\n',scores)
    mean_scores=np.mean(scores)
    print('----교차검증(cv)결과 평균----\n',mean_scores)
    return scores, mean_scores 
    
    ## 함수의 결과로 여러개의 데이터를 return할 수 있다.
    ## 튜플로 묶어서 반환하게 된다.(scores, mean_scores)

In [39]:
model_names=['KNeighborsClassifier','SVC','DecisionTreeClassifier']
sc_list=[]
mean_sc_list=[]
for model in model_list:
    sc, mean_sc=print_cv(model, X_train, y_train, 5)
    sc_list.append(sc)
    mean_sc_list.append(mean_sc)
print('***************************************')
print('----전체 모델 리스트----')
print(model_names)
print('----전체 모델의 교차검증 결과 리스트----')
print(sc_list)
print('----전체 모델의 교차검증 결과 평균----')
print(mean_sc_list)

----교차검증(cv)결과 리스트----
 [0.56606061 0.57090909 0.57818182 0.5769697  0.56727273]
----교차검증(cv)결과 평균----
 0.5718787878787879
----교차검증(cv)결과 리스트----
 [0.66181818 0.66181818 0.66181818 0.66181818 0.6630303 ]
----교차검증(cv)결과 평균----
 0.6620606060606061
----교차검증(cv)결과 리스트----
 [0.5769697  0.55878788 0.58181818 0.56363636 0.56121212]
----교차검증(cv)결과 평균----
 0.5684848484848485
***************************************
----전체 모델 리스트----
['KNeighborsClassifier', 'SVC', 'DecisionTreeClassifier']
----전체 모델의 교차검증 결과 리스트----
[array([0.56606061, 0.57090909, 0.57818182, 0.5769697 , 0.56727273]), array([0.66181818, 0.66181818, 0.66181818, 0.66181818, 0.6630303 ]), array([0.5769697 , 0.55878788, 0.58181818, 0.56363636, 0.56121212])]
----전체 모델의 교차검증 결과 평균----
[0.5718787878787879, 0.6620606060606061, 0.5684848484848485]


## svc

In [40]:
def grid_search(model, params, k):
    gs_model=GridSearchCV(model, params, cv=k)
    ## gs_model은 최적의 파라미터가 적용된 모델의 객체!
    gs_model.fit(X_train, y_train)
    print(gs_model.best_estimator_) ## 최적의 파라미터가 적용된 모델 객체
    print(gs_model.best_score_) ## 최적의 파라미터가 적용된 모델로 학습한 정확도 결과(acc)
    print(gs_model.best_params_) ## 그리드 서치를 통해 찾은 최적의 파라미터
    print(gs_model) ## grid_search 객체

In [41]:
params={ 
    'C': [0.001, 0.01, 0.1, 1, 10, 25],
    'random_state':[5, 50, 99]
}

In [42]:
grid_search(svm, params, 5)

SVC(C=1, random_state=5)
0.6620606060606061
{'C': 1, 'random_state': 5}
GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 25],
                         'random_state': [5, 50, 99]})


In [43]:
gs_svm=SVC(C=1, random_state=5)
gs_svm.fit(X_train, y_train)

SVC(C=1, random_state=5)

In [44]:
def test_validation(model, X_test, y_test):
    predict_result=model.predict(X_test)
    acc=accuracy_score(y_test, predict_result)
    print('accuracy >>', acc)
    cf=confusion_matrix(y_test, predict_result)
    print('confusion matrix >>\n', cf)
    ## confusion matrix는 binary classfication인 경우에는 보기가 괜찮으나,
    ## multi class classification인 경우에는 보기가 힘들다.
    ## confusion matrix를 가독성 좋에 볼 수 있는 것이 classification_report
    c_report=classification_report(y_test, predict_result)
    print('classification_report >>\n', c_report)

In [45]:
test_validation(gs_svm, X_test, y_test)

accuracy >> 0.6482558139534884
confusion matrix >>
 [[  0   0   0 128]
 [  0   0   0 170]
 [  0   0   0  64]
 [  0   1   0 669]]
classification_report >>
               precision    recall  f1-score   support

    식도락,맛집투어       0.00      0.00      0.00       128
        엑티비티       0.00      0.00      0.00       170
       축제,관광       0.00      0.00      0.00        64
      힐링,호캉스       0.65      1.00      0.79       670

    accuracy                           0.65      1032
   macro avg       0.16      0.25      0.20      1032
weighted avg       0.42      0.65      0.51      1032



## 설문 결과 predict

In [46]:
predict_col_list=['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']

In [47]:
len(predict_col_list)

18

In [48]:
def find_index(str):
    for i in range(len(predict_col_list)):
        if predict_col_list[i]==str:
            print(str+'의 index >> ',i)

In [49]:
find_index('age')

age의 index >>  8


In [50]:
df[['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']].head(1)

,sleepnights,accommodation_1,accommodation_2,accommodation_3,number_of_ppl,number_of_children,cost_total_trip,sex,age,month,living_area,transportation1,transportation2,transportation3,reason1,reason2,reason3,relationship
0,2,1,1,0,3,1,276000,2,38,5,11,1,0,0,14,1,11,1


In [51]:
df[['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']].head().values

array([[     2,      1,      1,      0,      3,      1, 276000,      2,
            38,      5,     11,      1,      0,      0,     14,      1,
            11,      1],
       [     2,      1,      1,      0,      3,      0, 318000,      2,
            48,      5,     11,      1,      0,      0,      1,     11,
             9,      1],
       [     1,     11,      0,      0,      6,      0, 107000,      1,
            19,      6,     11,      6,      0,      0,      4,     14,
             5,      3],
       [     0,      0,      0,      0,      3,      0,  50000,      2,
            55,      6,     11,      2,      0,      0,      1,      5,
             8,      3],
       [     2,      5,      5,      0,      4,      0, 233000,      1,
            29,      2,     11,      1,      0,      0,     13,     14,
             3,      1]])

In [52]:
df[['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']][df['target']=='엑티비티'].head().values

array([[     1,     11,      0,      0,      6,      0, 107000,      1,
            19,      6,     11,      6,      0,      0,      4,     14,
             5,      3],
       [     0,      0,      0,      0,      3,      1,  42000,      1,
            37,      6,     11,      1,      0,      0,      8,      1,
             3,      1],
       [     0,      0,      0,      0,      4,      0, 100000,      1,
            68,      8,     11,      1,      0,      0,     10,     11,
            14,      5],
       [     0,      0,      0,      0,      4,      2,  78048,      2,
            36,     12,     11,      1,      0,      0,     10,     14,
             8,      1],
       [     0,      0,      0,      0,      4,      0, 120000,      1,
            16,      8,     11,      6,      0,      0,     11,      0,
             0,      3]])

In [53]:
gs_svm.predict([[     1,     11,      0,      0,      6,      0, 107000,      1,
            19,      6,     11,      6,      0,      0,      4,     14,
             5,      3]])

array(['힐링,호캉스'], dtype=object)

In [54]:
df[['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']][df['target']=='식도락,맛집투어'].head().values

array([[    0,     0,     0,     0,     6,     0, 70000,     1,    30,
           12,    11,     2,     5,     0,     8,     4,     0,     3],
       [    0,     0,     0,     0,     4,     0, 97000,     1,    28,
            5,    11,     5,    10,     0,     5,     9,     3,     3],
       [    0,     0,     0,     0,     5,     1, 45000,     2,    37,
            1,    11,     1,     0,     0,     8,     5,    14,     1],
       [    0,     0,     0,     0,     4,     0, 40000,     1,    48,
            5,    11,     1,     0,     0,     8,    14,     0,     3],
       [    0,     0,     0,     0,     3,     1, 20000,     2,    38,
            6,    11,     1,     0,     0,     1,     8,     4,     1]])

## pickle, joblib

In [55]:
import pickle
import joblib

In [58]:
joblib.dump(gs_svm, 'trip_gs_svm.pkl')

['trip_gs_svm.pkl']

In [59]:
loaded_model = joblib.load('trip_gs_svm.pkl')
loaded_model

SVC(C=1, random_state=5)

In [60]:
loaded_model.predict([[    0,     0,     0,     0,     4,     0, 40000,     1,    48,
            5,    11,     1,     0,     0,     8,    14,     0,     3]])

array(['힐링,호캉스'], dtype=object)

## 설문지 답변 전처리, Spring에서 get방식으로 Django에 넘겨줌

In [61]:
answer1='35' # 나이 
answer2='1' # 성별
answer3='9' # 여행 월
answer4='1' # 거주지역
answer5='3' # 여행일수(박)
answer6='4,7,11' # 희망 숙박시설 1~3지망
answer7='3' # 여행인원
answer8='1' # 어린이 수
answer9='2' # 여행 인원간 관계
answer10='3,4,5' # 여행시 중요시 여기는 점 1~3순위
answer11='500000' # 총 예상 비용
answer12='2,5,6' # 주요 교통수단 1~3순위

In [62]:
answer_list=[0 for i in range(18)]

In [67]:
answer_list

[3, 4, 7, 11, 3, 1, 500000, 1, 35, 9, 1, 2, 5, 6, 3, 4, 5, 2]

In [63]:
len(answer_list)

18

In [64]:
answer6.split(',')[0]

'4'

In [65]:
answer_list[0]=int(answer5)
answer_list[1]=int(answer6.split(',')[0])
answer_list[2]=int(answer6.split(',')[1])
answer_list[3]=int(answer6.split(',')[2])
answer_list[4]=int(answer7)
answer_list[5]=int(answer8)
answer_list[6]=int(answer11)
answer_list[7]=int(answer2)
answer_list[8]=int(answer1)
answer_list[9]=int(answer3)
answer_list[10]=int(answer4)
answer_list[11]=int(answer12.split(',')[0])
answer_list[12]=int(answer12.split(',')[1])
answer_list[13]=int(answer12.split(',')[2])
answer_list[14]=int(answer10.split(',')[0])
answer_list[15]=int(answer10.split(',')[1])
answer_list[16]=int(answer10.split(',')[2])
answer_list[17]=int(answer9)
# predict할 list에 맞게 전처리

In [66]:
loaded_model.predict([answer_list])

array(['힐링,호캉스'], dtype=object)

In [73]:
np.__version__

'1.22.4'